# Notebook to combine TensorMap phases 
__Written by Haixing Fang, Jon Wright and James Ball__  
__Date: 21/02/2025__

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

# python environment stuff
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# dataset file to import
dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

phase_strs = ['Fe', 'Au']

# whether or not we are combining refined tensormaps (changes where we look for them)
combine_refined = True

In [ ]:
import os

import ImageD11.sinograms.dataset
from ImageD11.sinograms.tensor_map import TensorMap

%matplotlib ipympl

# Load data
## Dataset

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_path)
print(ds)

## Phases
If the parameter file was a json, we can access the unit cells via `ds.phases.unitcells`

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# what phases are we merging?
print(*[ds.phases.unitcells[phase_str].lattice_parameters for phase_str in phase_strs], sep='\n')

Choose where to import your TensorMaps from.  
If you refined them, you'll need to change the below paths to point to the separate refined tensormap h5 files.

In [ ]:
if combine_refined:
    tensor_maps = [TensorMap.from_h5(os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_refined_tmap_{phase_str}.h5')) for phase_str in phase_strs]
else:
    tensor_maps = [TensorMap.from_h5(ds.grainsfile, h5group='TensorMap_' + phase_str) for phase_str in phase_strs]

In [ ]:
try:
    for tmap in tensor_maps:
        tmap.plot('labels')
except KeyError:
    # no labels field
    pass

In [ ]:
tensor_map_combined = TensorMap.from_combine_phases(tensor_maps)

In [ ]:
tensor_map_combined.plot('phase_ids')
try:
    tensor_map_combined.plot('labels')
except KeyError:
    # no labels field
    pass
tensor_map_combined.plot('ipf_z')

# Export

In [ ]:
if combine_refined:
    tensor_map_combined.to_h5(os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_refined_tmap_combined.h5'))
    tensor_map_combined.to_paraview(os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_refined_tmap_combined.h5'))
else:
    tensor_map_combined.to_h5(os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_tmap_combined.h5'))
    tensor_map_combined.to_paraview(os.path.join(ds.analysispath, f'{ds.sample}_{ds.dset}_tmap_combined.h5'))